In [420]:
import altair as alt
import pandas as pd

In [405]:
def chart_settings(scheme_name, dict_schemes, continuous):
    # predefined scheme handling      
    if type(dict_schemes[scheme_name]) is not list:
        if continuous:
            no_colors_in_scheme = 300        
        else:
            no_colors_in_scheme = dict_schemes[scheme_name]
        df = pd.DataFrame({"i": range(no_colors_in_scheme)})
        custom_scheme = False

    # custom scheme handling
    else:
        if continuous:
            no_colors_in_scheme = 300
            df = pd.DataFrame({"i": range(no_colors_in_scheme)})  
        else:    
            no_colors_in_scheme = len(dict_schemes[scheme_name])
            colors_in_scheme = dict_schemes[scheme_name]
            df = pd.DataFrame({"i": range(no_colors_in_scheme), "hex": colors_in_scheme})
        custom_scheme = True

    # static height
    h = 50

    # width is dependent on no. of colors in scheme
    if continuous:
        w = 2
    elif no_colors_in_scheme <= 5:
        w = 50
    elif (no_colors_in_scheme > 5) and (no_colors_in_scheme <= 12):
        w = 30
    else:
        w = 15
    # cornerRadius dependent on height-width ratio
    cr_l = 0 if continuous else int(w / 2.4)
    cr_s = 0 if continuous else 5

    return df, w, h, cr_l, cr_s, no_colors_in_scheme, custom_scheme


def plot_scheme(scheme_name, dict_schemes, continuous=False, cvd=False):
    df, w, h, cr_l, cr_s, no, cs = chart_settings(scheme_name, dict_schemes, continuous)

    # define internal color scale definition    
    if cs and continuous:
        expr_hex = "scale('color', datum.i)"
        color_type = "quantitative"
    elif cs and not continuous:
        expr_hex = "datum.hex"
        color_type = "ordinal"
    elif not cs:
        expr_hex = "scale('color', datum.i)"
        color_type = "ordinal"
    scale_scheme = alt.Scale(range=dict_schemes[scheme_name]) if cs else alt.Scale(scheme=scheme_name)

    color_scale = (
        alt.Chart(alt.sequence(0, no, as_="i"))
        .mark_rect(height=alt.expr("0"), width=alt.expr("0"))
        .encode(alt.Color("i", type=color_type).scale(scale_scheme).legend(None))
    )    

    # define styling axis
    my_y_axis = alt.Axis(
        grid=False,
        ticks=False,
        labels=False,
        domain=False,
        titleAngle=0,
        titleAlign="right",
        titleBaseline="middle",
    )
    
    normal_vision = (
        alt.Chart(df, height=h, width=alt.Step(w))
        .transform_calculate(
            hex=expr_hex,
            rgb="rgb(datum.hex)",
            lum="luminance(datum.hex)",
        )
        .mark_rect(cornerRadius=cr_l, discreteBandSize=w)
        .encode(
            x=alt.X("i:O").axis(None),
            y=alt.Y("normal vision:N").axis(my_y_axis),
            fill=alt.Fill("hex", type=color_type).scale(None).legend(None),
            stroke=alt.Stroke("hex", type=color_type).scale(None).legend(None),
            tooltip=[
                alt.Tooltip("hex:O"),
                alt.Tooltip("rgb:O"),
                alt.Tooltip("lum:O", format=".2"),
            ],
        )
    )

    if not cvd:
        chart_comb = (
            (normal_vision & color_scale)
            .configure_concat(spacing=1)
            .configure_view(stroke=None)
        )
        return chart_comb


    green_blindness = (
        alt.Chart(df, height=20, width=alt.Step(w))
        .transform_calculate(
            hex=expr_hex,
            rgb="rgb(datum.hex)",
            gb_r="pow((4211+0.667*pow(datum.rgb['g'], 2.2)+0.2802*pow(datum.rgb['r'], 2.2)),1/2.2)",
            gb_g="pow((4211+0.667*pow(datum.rgb['g'], 2.2)+0.2802*pow(datum.rgb['r'], 2.2)),1/2.2)",
            gb_b="pow((4211+0.95724*pow(datum.rgb['b'], 2.2)+0.02138*pow(datum.rgb['b'], 2.2)-0.02138*pow(datum.rgb['r'], 2.2)),1/2.2)",
            greenblind_rgb="rgb(datum.gb_r,datum.gb_g,datum.gb_b)",
        )
        .mark_rect(cornerRadius=cr_s)
        .encode(
            x=alt.X("i:O").axis(None),
            y=alt.Y("green-blindness:N").axis(my_y_axis),
            fill=alt.Fill("greenblind_rgb", type=color_type).scale(None).legend(None),
            stroke=alt.Stroke("greenblind_rgb", type=color_type).scale(None).legend(None),            
        )
    )

    red_blindness = (
        alt.Chart(df, height=20, width=alt.Step(w))
        .transform_calculate(
            hex=expr_hex,
            rgb="rgb(datum.hex)",
            rb_r="pow((782.7+0.8806*pow(datum.rgb['g'], 2.2)+0.1115*pow(datum.rgb['r'], 2.2)),1/2.2)",
            rb_g="pow((782.7+0.8806*pow(datum.rgb['g'], 2.2)+0.1115*pow(datum.rgb['r'], 2.2)),1/2.2)",
            rb_b="pow((782.7+0.992052*pow(datum.rgb['b'], 2.2)-0.03974*pow(datum.rgb['b'], 2.2)+0.003974*pow(datum.rgb['r'], 2.2)),1/2.2)",
            redblind_rgb="rgb(datum.rb_r,datum.rb_g,datum.rb_b)",
        )
        .mark_rect(cornerRadius=cr_s)
        .encode(
            x=alt.X("i:O").axis(None),
            y=alt.Y("red-blindness:N").axis(my_y_axis),
            fill=alt.Fill("redblind_rgb", type=color_type).scale(None).legend(None),
            stroke=alt.Stroke("redblind_rgb", type=color_type).scale(None).legend(None),             
        )
    )

    grayscale = (
        alt.Chart(df, height=20, width=alt.Step(w))
        .transform_calculate(
            hex=expr_hex,
            rgb="rgb(datum.hex)",
            lumn_rgb="datum.rgb['r']*0.3+datum.rgb['g']*0.59+datum.rgb['b']*0.3",
            grayscale_rgb="rgb(datum.lumn_rgb,datum.lumn_rgb,datum.lumn_rgb)",
        )
        .mark_rect(cornerRadius=cr_s)
        .encode(
            x=alt.X("i:O").axis(None),
            y=alt.Y("grayscale:N").axis(my_y_axis),
            fill=alt.Fill("grayscale_rgb", type=color_type).scale(None).legend(None),
            stroke=alt.Stroke("grayscale_rgb", type=color_type).scale(None).legend(None),               

        )
    )



    chart_comb = (
        (normal_vision & green_blindness & red_blindness & grayscale & color_scale)
        .configure_concat(spacing=1)
        .configure_view(stroke=None)
    )
    return chart_comb


In [421]:
tol_schemes = {
    "tol_bright" : ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB'],
    "tol_highcontrast" : ['#FFFFFF', '#DDAA33', '#BB5566', '#004488', '#000000'],
    "tol_vibrant": ['#0077BB', '#33BBEE', '#009988', '#EE7733', '#CC3311', '#EE3377', '#BBBBBB'],
    "tol_muted": ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', '#DDCC77' ,'#CC6677' , '#882255', '#AA4499', '#DDDDDD'],
    "tol_mediumcontrast": ['#FFFFFF', '#EECC66', '#994455','#6699CC', '#997700', '#994455', '#004488', '#000000'],
    "tol_pale": ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD'],
    "tol_dark": ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555'],
    "tol_light": ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33', '#AAAA00', '#EEDD88', '#EE8866', '#FFAABB', '#DDDDDD']
}

In [422]:
catg_schemes = {
    "accent": 8,
    "category10": 10,
    "category20": 20,
    "category20b": 20,
    "category20c": 20,
    "dark2": 8,
    "paired": 12,
    "pastel1": 9,
    "pastel2": 8,
    "set1": 9,
    "set2": 8,
    "set3": 12,
    "tableau10": 10,
    "tableau20": 20,
}

In [397]:
plot_scheme('spectral', divg_schemes, continuous=False, cvd=True)

cs:False, cont:False
ct:ordinal


alt.VConcatChart(...)

In [398]:
plot_scheme('tol_bright', tol_schemes)

cs:True, cont:False
ct:ordinal


alt.VConcatChart(...)

In [403]:
plot_scheme("non-uniform", percep_schemes, continuous=True)

cs:True, cont:True
ct:quantitative


alt.VConcatChart(...)

In [404]:
plot_scheme("uniform", percep_schemes, continuous=True)

cs:True, cont:True
ct:quantitative


alt.VConcatChart(...)

In [383]:
divg_schemes = {
    "blueorange": 9,
    "brownbluegreen": 9,
    "purplegreen": 9,
    "pinkyellowgreen": 9,
    "purpleorange": 9,
    "redblue": 9,
    "redgrey": 9,
    "redyellowblue": 9,
    "redyellowgreen": 9,
    "spectral": 9,
}

In [384]:
percep_schemes = {
    "uniform": ["#542788", "#1f7abd", "#4dbd33", "#fdae61", "#d7191c"],
    "non-uniform": ["#FF0000", "#FF6600", "#FFFF00", "#00FF00", "#0000FF"]
}

In [417]:
color_vision_deficiency = {
    'unfriendly':['#006400', '#228B22', '#00FF00', '#7FFF00', '#FFFF00', '#FFA500', '#FF0000'],
    'tol_sunset_friendly': ['#364B9A', '#4A7BB7', '#6EA6CD', '#98CAE1', '#C2E4EF', '#EAECCC', '#FEDA8B', '#FDB366', '#F67E4B', '#DD3D2D', '#A50026']
}

In [418]:
plot_scheme("unfriendly", color_vision_deficiency, cvd=True, continuous=True)

alt.VConcatChart(...)

In [419]:
plot_scheme("tol_sunset_friendly", color_vision_deficiency, cvd=True, continuous=True)

alt.VConcatChart(...)